In [1]:
from weave_example_demo.llm_types.prompts import PromptTemplate
from weave_example_demo.llm_types.rag.vector_store import VectorStore
from weave_example_demo.scorers.llm_guard_scorer import LLMGuardScorer
from weave_example_demo.scorers.tonic_validate_scorer import TonicValidateScorer
from weave_example_demo.llm_types.rag.rag import RAGModel


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import weave

In [3]:
weave.init('bioasq-rag-data')
qap = weave.ref('QuestionAnswerPairsTrainFiltered:latest').get()
vector_store = weave.ref('VectorStore:latest').get()
embedding_model = weave.ref('SentenceTransformersModel:latest').get()



Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/bioasq-rag-data/weave


In [4]:
embedding_model.model



"SentenceTransformer(\n  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})\n  (2): Normalize()\n)"

In [5]:
vector_store.set_embedding_model(embedding_model)

🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/2d0ff348-dbb5-4f53-9277-9068646e0a74


In [6]:
_context = vector_store._get_chebyshev_distances("What is MSR", n=2)
_context = vector_store.get_most_relevant_documents("What is MSR", n=2)


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/59838038-68c1-4993-9c96-2db6dff132f9


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/0252e077-1083-45d4-a174-e046154e7959


In [7]:
_context

[{'document': TraceDict({'passage': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', 'hcSV0sXeRml9nrqnlNkVDwxJygi06pB5xEDF6XVF1IE', 'key', 'passage']), 'id': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', 'hcSV0sXeRml9nrqnlNkVDwxJygi06pB5xEDF6XVF1IE', 'key', 'id'])}),
  'score': 0.24590397466713787},
 {'document': TraceDict({'passage': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', '3vG4sryOI0bbWxXqxGxrUpIOYkRiTxD1DHzb8Cu8dKs', 'key', 'passage']), 'id': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', '3vG4sryOI0bbWxXqxGxrUpIO

In [8]:
system_prompt = """
### Instruction ###
You are an expert medical assistant. Your task is to provide accurate and concise answers to medical questions based on the provided data. Ensure that your responses are clear, informative, unbiased, and avoid relying on stereotypes. Answer in a natural, human-like manner. Here are some examples:

### Example ###
1. Which enzyme deficiency can cause GM1 gangliosidoses?
   - GM1 gangliosidoses are associated with deficiency of β-galactosidase.

2. Which are the drugs utilized for the burning mouth syndrome?
   - Dopaminergic drugs should be given in patients with BMS. Catuama reduces the symptoms of BMS and may be a novel therapeutic strategy for the treatment of this disease. Capsaicin, alpha-lipoic acid (ALA), and clonazepam were those that showed more reduction in symptoms of BMS. Treatment with placebos produced a response that was 72% as large as the response to active drugs.

3. Describe a diet that reduces the chance of kidney stones.
   - People can help prevent kidney stones by making changes in fluid intake and, depending on the type of kidney stone, changes in consumption of sodium, animal protein, calcium, and oxalate. Drinking enough fluids each day is the best way to help prevent most types of kidney stones. Health care providers recommend that a person drink 2 to 3 liters of fluid a day. People with cystine stones may need to drink even more. Though water is best, other fluids may also help prevent kidney stones, such as citrus drinks.

4. Does smoking increase risk for glioblastoma?
   - No. Smoking does not increase risk for glioblastoma.

5. Inhibition of which enzyme is the mechanism of action of alisertib?
   - Alisertib (MLN8237) is a selective Aurora kinase inhibitor that acts by interfering with spindle organization and chromosome alignment during mitosis. It has been tested in patients with gastric cancer, breast cancer, relapsed and refractory aggressive B- and T-cell non-Hodgkin lymphomas, epithelial ovarian, fallopian tube, and primary peritoneal carcinoma.

### Instruction ###
Provide your answers in a similar format. Ensure that your answer is unbiased and avoids relying on stereotypes.
"""

In [9]:
human_prompt = """
### Question ###
{question}

### Contexts ###
{context}

### Answer ###
"""



In [10]:
bioasq_rag = RAGModel(
    # vector_store=vector_store,
    system_prompt=system_prompt,
    human_prompt=human_prompt
)
bioasq_rag.set_vector_store(vector_store) #TODO: Allow for vector store to be passed in as an argument



🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/cc20a298-6914-451e-88af-16766b95d7ec


In [11]:
bioasq_rag.predict("What is MSR")

/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/ea8c704f-f217-4c24-8005-aba81a1d51b2


{'answer': 'MSR stands for Motor Neurone Disease, which is also known as amyotrophic lateral sclerosis (ALS). It is a serious progressive neurological disorder characterized by the loss of upper motor neurons (UMN) and lower motor neurons (LMN). The exact cause of the disease is still unknown, and there is currently no effective treatment available. Various theories exist regarding the pathogenesis of the disease, including abnormalities in neurotransmitters, neuropeptides, gene expression, glutamate metabolism, and more. Research is ongoing to understand the mechanisms underlying the disease and to develop potential treatments.',
 'context': [{'document': TraceDict({'passage': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', 'hcSV0sXeRml9nrqnlNkVDwxJygi06pB5xEDF6XVF1IE', 'key', 'passage']), 'id': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFilt

In [12]:
from weave_example_demo.llm_types.agent.tools import tool_registry, RAGTool
from weave_example_demo.llm_types.agent.agent import LLMAgentModel

In [13]:
tool_registry.add_tool(
    # key="BioASQRAGTool",
    key="RAGTool",
    function=RAGTool(rag_model=bioasq_rag),
    description="Generate a medical response using the BioASQ RAG model",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The medical question for the BioASQ RAG model",
            },
            "n_documents": {
                "type": "integer",
                "description": "The number of medical documents to retrieve",
                "default": 2,
            },
        },
        "required": ["question"],
    },
)

In [14]:
bioasq_agent_system_prompt = (
    "### Instruction ###\n"
    "You are an intelligent assistant named BioASQ-Bot, an expert in biomedical question answering. Your task is to provide accurate, relevant, and clear answers to biomedical questions using the information from the provided tools and context.\n\n"
    "### Tools ###\n"
    "You have access to the following tools:\n"
    "{tools}\n\n"
    "### Process ###\n"
    "Follow this step-by-step process to answer the question:\n"
    "1. Analyze the question to identify the key information needed.\n"
    "2. Use the available tools to retrieve relevant information.\n"
    "3. Synthesize the retrieved information into a coherent answer.\n"
    "4. Cite the sources used to formulate your answer.\n"
    "5. If the question is unclear or lacks context, ask for clarification.\n"
    "6. Provide the final answer in a clear, unbiased, and professional manner.\n\n"
    "### Format ###\n"
    "Use the following format for your response:\n"
    "Question: <The input question you must answer>\n"
    "Thought: <Your analysis of the question and plan to answer it>\n"
    "Action: <The action to take, should be one of the provided tools>\n"
    "Action Input: <The input to the selected action>\n"
    "Observation: <The result of the action>\n"
    "... (Repeat Thought/Action/Action Input/Observation as needed)\n"
    "Thought: <Your reflection on whether you can now answer the question>\n"
    "FINISH: <The final answer to the original question>\n\n"
    "### Evaluation ###\n"
    "Your performance will be evaluated based on:\n"
    "- Accuracy and relevance of your answers\n"  
    "- Clarity and coherence of your explanations\n"
    "- Proper citation of sources\n"
    "- Ability to ask for clarification when needed\n"
    "- Professionalism and objectivity in your responses\n\n"
    "### Instruction ###\n"
    "You must start your final answer with 'FINISH:'. Do not provide any information that is not supported by the tools and context provided. If you cannot answer the question based on the available information, say 'FINISH: I do not have enough information to answer this question.' Let's begin!\n"
)

bioasq_agent_human_prompt = "Question: {question}\n" "Response:"



In [15]:
tools = [
        #  "RAGTool" #BUG: Slice issue
    ]
agent = LLMAgentModel(
    model_name="gpt-3.5-turbo",
    tools=tools,
    human_prompt=bioasq_agent_human_prompt,
    system_prompt=bioasq_agent_system_prompt,
)



In [16]:
agent.predict("What is MSR? Generate a medical response using the BioASQ RAG model")

Message(content='Thought: The question is asking about the definition or explanation of "MSR" in a medical context. I will use the BioASQ RAG model to generate a medical response for this question.\n\nAction: Utilize the BioASQ RAG model to generate a medical response.\nAction Input:\n- Question: "What is MSR?"\n- Number of medical documents to retrieve: 1\n\nObservation: The response generated by the BioASQ RAG model will provide the information about MSR in a medical context.\n\nFINISH: The response generated by the BioASQ RAG model will provide the information about MSR in a medical context.', role='assistant')
False
Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"question":"What is MSR?","n_documents":1}', name='RAGTool'), id='call_fT3Ga9y9OQdgiaSVbl7thxc2', type='function')])
True


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Message(content='FINISH: MSR stands for Matuzumab, which is a fully humanized epidermal growth factor receptor (EGFR)-specific monoclonal antibody developed by Merck KGaA. It is being investigated as a potential treatment for EGFR-bearing tumors and is currently in phase II clinical trials for gastric, cervical, pancreatic, and ovarian cancers.', role='assistant')
False
🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/2f4b5e45-d217-4483-bc30-ce6852364c61


'FINISH: MSR stands for Matuzumab, which is a fully humanized epidermal growth factor receptor (EGFR)-specific monoclonal antibody developed by Merck KGaA. It is being investigated as a potential treatment for EGFR-bearing tumors and is currently in phase II clinical trials for gastric, cervical, pancreatic, and ovarian cancers.'